In [3]:
import ipywidgets as widgets
from algorithm import pipeline
import cv2
from io import BytesIO
import numpy as np
from matplotlib import pyplot as plt

In [2]:
fileselect = widgets.FileUpload(
    accept = 'image/*',
    multiple = False,
)

read_file = widgets.Button(
    description = 'Load image'
)

compactness_input = widgets.IntText(
    value = 20,
    description = "compactness",
    disabled = False,
)

n_seg_input = widgets.IntText(
    value = 2,
    description = "# segments",
    disabled = False,
)

sigma_input = widgets.IntText(
    value = 0,
    description = "sigma",
    disabled = False,
)

n_samples_input = widgets.IntText(
    value = 2,
    description = '# samples',
    disabled = False,
)

go_button = widgets.Button(
    description = 'Calculate',
)

controls = widgets.VBox([
    widgets.HBox([fileselect, read_file]),
    n_seg_input,
    compactness_input,
    sigma_input,
    n_samples_input,
    go_button,
])

as_imported = widgets.Output()
SLIC_out = widgets.Output()
background_out = widgets.Output()

results_out = widgets.VBox([
    sample_number := widgets.Dropdown(
        options = [],
    ),
    n_pixels_label := widgets.Label(),
    view_foreground := widgets.Checkbox(
        description = 'View mask'
    ),
    sample_output := widgets.Output(),
])

views = widgets.Tab()
views.children = [as_imported, SLIC_out, background_out, results_out]
for i, title in enumerate(['Input', 'SLIC', 'Background', 'Results']):
    views.set_title(i, title)

img = None
results = None
    
def load_image(*nada):
    global img
    fname = next(iter(fileselect.value))
    data = fileselect.value[fname]['content']
    img = cv2.imdecode(
        np.frombuffer(BytesIO(data).read(), 
                      dtype=np.uint8), 
        1)
    fileselect.value.clear()
    fileselect._counter = 0
    as_imported.clear_output()
    with as_imported:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()

def callback(*nada):
    global results
    results = pipeline(img,
                       compactness=compactness_input.value,
                       n_segments=n_seg_input.value,
                       sigma=sigma_input.value,
                      )
    SLIC_out.clear_output()
    with SLIC_out:
        plt.imshow(results['slic'])
        plt.show()
    background_out.clear_output()
    with background_out:
        plt.imshow(results['background'])
        plt.show()
    sample_number.options = [('Coin',0)] + [(f'Sample {n}', n+1) 
                                        for n in range(n_samples_input.value)]

def set_sample(change):
    index = sample_number.value
    n_pixels_label.value = '# pixels: ' + str(results['regions'][index].filled_area)
    sample_output.clear_output()
    with sample_output:
        region = results['regions'][index]
        if view_foreground.value:
            plt.imshow(region.filled_image)
        else:
            plt.imshow(cv2.cvtColor(img[region.slice], cv2.COLOR_BGR2RGB))
        plt.show()
        
def on_init():
    with as_imported:
        print('Import an image by: \n 1) Uploading it. \n 2) Clicking "Load image".')
        print('Then select parameters and calculate.')
    init_msg = 'These tabs will be filled\n after pressing calculate.'
    with SLIC_out:
        print(init_msg)
    with background_out:
        print(init_msg)
    with sample_output:
        print(init_msg)
    
        
on_init()
    
read_file.on_click(load_image)
go_button.on_click(callback)
sample_number.observe(set_sample, names='value')
view_foreground.observe(set_sample)


display(widgets.HBox([controls, views]))
        